<div align="center">

# **Alejandro Coman Venceslá**

### Doble Grado en Ingeniería Informática y  
### Administración y Dirección de Empresas
#### Universidad de Granada

<br>

<div align="center">
  <img src="https://etsiit.ugr.es/sites/centros/etsiit/public/template-extra/etsiit-logo.png" alt="Imagen 1" style="width: 200px; margin-right: 40px;">
  <img src="https://etsiit.ugr.es/sites/centros/etsiit/public/color/ugr-41cc9222/logo-mono.svg" alt="Imagen 2" style="width: 300px; margin-left: 40px; margin-bottom: 60px">
</div>

**Trabajo de Fin de Grado**

<br><br>

*Análisis de sesgos en modelos de inteligencia artificial generativa textual.*

</div>

# Capítulo 2.1. Generación de biografías

Este cuaderno contiene el código empleado para el segundo capítulo 

In [1]:
import string
import pandas as pd
import requests
import time
import csv
from openai import OpenAI
from tqdm import tqdm
import itertools
import sys
import os
from pathlib import Path

# 1. Calcula la carpeta padre del notebook
parent_dir = Path().resolve().parent

# 2. Inserta esa ruta al principio de sys.path
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

# 3. Ahora ya puedes importar
from variables import *

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=API_KEY,
)


## Cargar el CSV de personajes

In [2]:
df_personajes = pd.read_csv("../Capítulo primero/frecuentes.csv")  # Cambia a la ruta correcta si es necesario
names = df_personajes['Name'].tolist()  # Extrae los nombres desde la columna "Name"

In [3]:
len(names)

5904

## Generación de biografías

### Función para hacer peticiones a la API

In [4]:
def obtener_biografia(nombre, modelo):
    prompt = f"Tell me a shortened version of {nombre}'s biography. Use only 300 words and highlight the most important events in the author's life, all in one paragraph."
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": modelo,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7
    }
    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        content = response.json()
        return content["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"Error with {modelo} for {nombre}: {e}")
        return "ERROR"

In [5]:
total = len(names)

cols = ['Name'] + modelos
if not os.path.exists("biografias_raw.csv"):
	pd.DataFrame(columns=cols).to_csv("biografias_raw.csv", index=False, encoding='utf-8')

# Empiezo por donde me quedé
df_out = pd.read_csv("biografias_raw.csv")
processed = df_out['Name'].tolist()
start_idx = len(processed)
names[start_idx]

'Jennifer Doudna'

In [6]:
for idx in range(start_idx, total):
    name = names[idx]
    row = {'Name': name}
    for modelo in modelos:
        try:
            bio = obtener_biografia(name, modelo)
        except Exception as e:
            print(f"Error with {modelo} for {name}: {e}")
            bio = ""
        row[modelo] = bio
        
	# Añado fila al CSV
    pd.DataFrame([row]).to_csv(
        "biografias_raw.csv", mode='a', header=False, index=False, encoding='utf-8', quoting=csv.QUOTE_ALL
	)
    
	# Imprimo el progreso
    print(f"Processed {idx + 1}/{total} for {name}")

Processed 5901/5904 for Jennifer Doudna
Processed 5902/5904 for Maria Chiara Carrozza
Processed 5903/5904 for Name 1
Processed 5904/5904 for María Blasco
